### Toy Relevance Vector Machine (Regression) Implementation

Author: [Evgenii Egorov](mailto:egorov.evgenyy@ya.ru)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We consider the following model:
    
\begin{equation*}
    \begin{aligned}
    & p(t_n|x_n, w;\beta) = \mathcal{N}(t_n| \textbf{w}^Tx_n, \beta^{-1}), \\
    & p(\textbf{t}|X,\textbf{w};\beta) = \prod\limits_{n=1}^{N}p(t_n|x_n, \textbf{w};\beta) = \mathcal{N}(\textbf{t}|X\textbf{w}, \beta^{-1}), \\
    & p(\textbf{w};\alpha) = \prod\limits_{d=1}^{D}\mathcal{N}(w_d|0, \alpha_d^{-1})=\mathcal{N}(\textbf{w}|0,A^{-1}).
    \end{aligned}
\end{equation*}  

And for optimize the evidence we come up with the following equations:

$$
\max\limits_{\alpha,\beta}\tfrac{N}{2}\log\beta+\tfrac{1}{2}\log|A|- \tfrac{1}{2}\log|\left(\beta X^TX+A\right)|-\tfrac{\beta}{2}\|\textbf{t}-X\mu\|_2^2 -\tfrac{1}{2}\mu^TA\mu.
$$

\begin{equation*}
\begin{aligned}
& \mu^{new} = \beta(\beta X^T X + A)^{-1}X^T\textbf{t},\\
& \alpha_i^{new} = \frac{1}{\mu^2_i}(1-\Sigma_{ii}^{old}\alpha_i^{old}),\\
& \beta^{new} = \frac{1}{\|t-X\mu\|_2^2}\left(N-\text{trace} (I-\Sigma^{old}A^{old})\right),\\
& \Sigma^{new} = (\beta^{new} X^TX + A^{new})^{-1}.\\
\end{aligned}
\end{equation*}

For details and derivations please see the corresponding slides: [slides](https://drive.google.com/open?id=1ezz6LXef6pFdbljSvvZIlnmYqietQg-s)

##### Task 1.1
Please write code at the each method <i>update_%smth%</i>.

In [4]:
class RVM:
    def __init__(self, data):
        self.X = data[0]
        self.t = data[1]
        
        self.alpha = np.array([0.1] * self.X.shape[1])
        self.beta = 1.
        self.Sigma = None
        
        self.data_cov = None
        self.data_mu = None
        
        self.idx_relevant = np.array(range(self.X.shape[1]))
        
    def data_params(self):
        data_cov = self.X.T @ self.X
        data_mu = self.X.T @ self.t
        return data_cov, data_mu
    
    def get_sigma(self):
        return np.linalg.inv(self.beta * self.data_cov + np.diag(self.alpha))
    
    @staticmethod
    def update_mu(Sigma, beta, data_mu):
        return beta * Sigma @ data_mu
    
    @staticmethod
    def update_alpha(Sigma, beta, mu, alpha):
        sigma_diag = np.diag(Sigma)
        return (1 - sigma_diag * alpha) / mu ** 2
    
    @staticmethod
    def update_beta(Sigma, alpha, mu, X, t):
        N = X.shape[0]
        sigma_diag = np.diag(Sigma)
        return (N - np.sum(1 - sigma_diag * alpha)) / np.linalg.norm(t - X @ mu) ** 2
    
    def update_step(self):
        mu = self.update_mu(self.Sigma, self.beta, self.data_mu)
        
        alpha = self.update_alpha(self.Sigma, self.beta, mu, self.alpha)
        beta = self.update_beta(self.Sigma, self.alpha, mu, self.X, self.t)
        return mu, alpha, beta
    
    def elbo(self):
        N = self.X.shape[0]
        lbound = N * np.log(self.beta) + np.sum(np.log((self.alpha))) - self.beta * np.linalg.norm(self.X @ self.mu - self.t) ** 2\
        - np.sum(self.mu ** 2 * self.alpha) + np.log(np.linalg.det(self.Sigma))
        return lbound
    
    def prune(self, threshold):
        idx = np.where(self.alpha < threshold)[0]
        if len(idx) != 0:
            bad = np.where(self.alpha >= threshold)[0]
            temp = self.idx_relevant[np.where(self.idx_relevant != -1)] 
            temp[bad] = -1
            self.idx_relevant[np.where(self.idx_relevant != -1)] = temp
            
            self.X = self.X[:, idx]
            self.alpha = self.alpha[idx]
            self.mu = self.mu[idx]
            self.data_cov, self.data_mu = self.data_params()
            
            
    def fit(self, prune=1e5, max_iter=10):
        self.data_cov, self.data_mu = self.data_params()
        
        for i in range(max_iter):
            self.Sigma = self.get_sigma()
            self.mu, self.alpha, self.beta = self.update_step()
            if prune is not None:
                self.prune(prune)
            else:
                self.alpha = np.clip(self.alpha, 0., 1e5)
            print(i, self.elbo())
        return 0
    
    def get_relevant_features_prune(self):
        return self.idx_relevant[np.where(self.idx_relevant != -1)]

Let's see how it works. We generate some sparse data.

In [5]:
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score

In [6]:
X, t, coef_true = make_regression(100, 100, 3, coef=True)
print(coef_true)
print(np.where(coef_true != 0)[0])

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 83.8110758   0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 63.64974908  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.         

And now we expect that RVM will return for us exactly these non-zero features.m

In [7]:
reg = RVM((X, t))

In [8]:
reg.fit(1e5)

0 -571.4852563595872
1 503.0053096111681
2 3668.457456244226
3 5895.977785780902
4 5817.316452570509
5 5785.002094980026
6 5775.732725762403
7 5871.9997629475765
8 5918.5623620195865
9 5814.520528609179


0

In [9]:
reg.get_relevant_features_prune()

array([30, 66, 94])

Eeee!